# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [1]:
# verifying pathlib is clean
import pathlib
print(pathlib.__file__)

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.2032.0_x64__qbz5n2kfra8p0\Lib\pathlib\__init__.py


In [2]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import socket
import pathlib
import time
import torch
import numpy as np

from robot.al5d_position_controller import PositionController, RobotPosition

hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    try:
        from robotcontrol.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxend module not found, cannot use gamepad")
        available_gamepad = False
    available_robot = True
elif hostname == "szenes.local":
    available_gamepad = False
    available_robot = False
elif hostname == "glassy":
    available_gamepad = False
    available_robot = False
else:
    available_gamepad = False
    available_robot = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot


from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

Hostname is glassy
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.2032.0_x64__qbz5n2kfra8p0\Lib\pathlib\__init__.py
***ExpRun**: Loading pointer config file:
	C:\Users\lotzi\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	c:\Users\lotzi\Work\_Config\BerryPicker\cfg\settings.yaml
Using device: cuda


In [3]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

Hostname is glassy
Path for external experiments: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
Path for external data: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Experiment config path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
***ExpRun**: Experiment data path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Experiment demonstration copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\robot_al5d
***ExpRun**: Experiment automate copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\automate
***ExpRun**: Experiment behavior_cloning copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\behavior_cloning
***Exp

In [4]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
run = "runbc_bc_lstm_0001"
exp = Config().get_experiment(experiment, run)

# note that we are loading the exprun for the robot controller even if the robot is not connected
exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])

if hostname == "glassy":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_glassy")
else:
    exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

['bc_0001',
 'bc_compare_00',
 'bc_lstm_00',
 'bc_lstm_0001',
 'bc_lstm_mdn_00',
 'bc_lstm_mdn_0001',
 'bc_lstm_residual_0001',
 'bc_lstm_resid_00',
 'bc_mlp_00',
 'bc_mlp_0001',
 'bc_run_00',
 'bc_run_0001',
 'bc_verify_lstm_00',
 'bc_verify_lstm_mdn_00',
 'bc_verify_lstm_resid_00',
 'bc_verify_mlp_00',
 'mdn_00',
 'mdn_for_bc_00',
 'runbc_bc_lstm_0001',
 'runbc_bc_lstm_mdn_0001',
 'runbc_bc_lstm_residual_0001',
 'runbc_bc_mlp_0001',
 '_defaults_behavior_cloning']
***ExpRun**: Configuration for exp/run: behavior_cloning/runbc_bc_lstm_0001 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_00 successfully loaded
***ExpRun**: Experiment default config C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\controllers\_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_glassy successfully loaded
***ExpRun**: Configuration for exp/run: behavior_cloning/bc_lstm_0001 successfully loaded
***E

In [5]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp, device)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp, device)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path))   

cap0 works


<All keys matched successfully>

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

In [6]:
def safe_robot_control(a, robot_controller, exp_robot_controller):
    """Moves the robot to the position, with various safety controler"""
    pos_target = RobotPosition.from_normalized_vector(exp_robot_controller,a)
    print(f"Target position: {pos_target}", flush=True)
    if not robot_controller:
        return "0"
    # we assume that we have the robot
    # ask the user whether we indeed want to continue moving?
    pos_current = robot_controller.get_position()
    distance = pos_current.empirical_distance(exp_robot_controller, pos_target)
    query = f"Next position: {pos_target} at distance={distance} Proceed with the move? (Y/N/Q)"
    ans = input(query)
    if ans.lower() == "y":
        robot_controller.move(pos_target)
        return "y"
    elif ans.lower() == "q":
        return "q"


In [7]:
def run_robot(model, robot_controller, exp_robot_controller, camera_controller, controller_interval = 0.1):
    """Robot running loop. Loads the image from the camera, runs the model, and executes the recommended code."""
    # last_interval = 0
    controller_interval = 0.1

    transform = sp_helper.get_transform_to_sp(exp_sp)
    # the backlog of the last sequence length steps
    z_seq = []

    while True:
        start_time = time.time() 
        # Let us assume that the first image here is the one
        toexit = camera_controller.update()
        if toexit:
            break
        image = camera_controller.images[exp["control_camera"]]
        # FIXME: this is totally won't work like this
        # there should be a notebook with the input processing???
        sensor_readings = sp_helper.load_capture_to_tensor(image, transform, device)
        z = sp.process(sensor_readings[0])
        print(z)
        if exp_bc["sequence_length"]: # no sequence processsing
            z_seq.append(z)
            if len(z_seq) > exp_bc["sequence_length"]:
                z_seq.pop(0)
            if len(z_seq) < exp_bc["sequence_length"]:
                ## we don't have enough in the queue, wait.
                print("Not acting, waiting on the queue", flush=True)
                z_tensor = None
                pass
            else:
                # FIXME: this should be applied online, not step by step
                z_seq_np = np.array(z_seq)
                z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(device)
        else:
            z_tensor = torch.tensor(z).unsqueeze(0).to(device)
        if z_tensor == None:
            continue
        if not model.stochastic:                
            a_pred = model(z_tensor)
        else:
            a_pred = model.forward_and_sample(z_tensor)

        retval = safe_robot_control(a_pred[0], robot_controller, exp_robot_controller)
        if retval == "q":
            break

        # ensuring that the execution happens 
        end_time = time.time() 
        execution_time = end_time - start_time 
        # last_interval = execution_time
        time_to_sleep = max(0.0, controller_interval - execution_time) 
        time.sleep(time_to_sleep) 

In [8]:
run_robot(model = model, robot_controller = robot_controller, exp_robot_controller = exp_robot_controller, camera_controller = camera_controller)

[  8.560643     0.18289916   5.199831    -0.7239417    1.6205193
   1.3338293   -4.954725    -2.3177917    8.281        4.7133417
  -3.159969    -2.0120237   10.097665    12.602397    -1.7889596
   0.28510958   1.8744603   -4.443973     2.356552    -3.7234652
   2.6208358    6.282378    -1.123206     4.976312     2.2027323
   4.4760394   -2.625364    -0.43475634  -1.4979088   11.135629
   0.545619     0.56999934  13.450756     1.5832227   -0.65762085
  -6.2940283    1.5091013   -7.826934    -9.530317    -6.6203604
   1.3326741   -2.6566637    5.4603424   -0.3546393    5.504454
  -4.4647994    2.0397627    0.16254625  -2.9706583   -0.7830983
   1.4835396    0.7489568   11.289201    -2.1692722   -8.786485
   4.9333434   -0.15167603  -0.791704    -2.5762897    6.235998
   5.056784    -1.193928     9.08522      0.73261225  -2.511566
  -8.383672    -1.2031765   -2.3478327    3.3131897   -1.6000885
   1.2078668    3.523915    -0.75276804  -2.557894     2.0628183
   6.731535    -4.8550096    

In [9]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()